In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import models
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import ImageFolder, DatasetFolder
from torch.utils.data.dataset import random_split
from PIL import Image
from torchvision.transforms import v2
import os

In [3]:
import os

class VocalPortraitDataset():
    # TODO: labels not implemented
    def __init__(self, root_path:str, transform:None):
        # root_path is the faces folder path
        self.root_path = root_path
        self.transform = transform
        self.image_paths = self.get_image_paths()

    def get_image_paths(self):
        image_paths = []
        
        person_folder = os.listdir(self.root_path)
        for person in person_folder:
            for nationality in ['English', 'Urdu']:
                if os.path.exists(f"{self.root_path}/{person}/{nationality}"):
                    video_ids = os.listdir(f"{self.root_path}/{person}/{nationality}")
                    for video_id in video_ids:
                        image_names = os.listdir(f"{self.root_path}/{person}/{nationality}/{video_id}")
                        for image_name in image_names:
                            image_paths.append(f"{self.root_path}/{person}/{nationality}/{video_id}/{image_name}")
        return image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        # label = self.labels[idx]
        return image, idx

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class EncoderBlock(nn.Module):
  def __init__(
    self,
    input_dim:tuple,
    latent_dim:int,
    out_channel:int=None,
  ):
    super().__init__()
    assert len(input_dim) == 3, "input_dim must be a tuple of 3 numbers (C, H, W)"
    
    self.in_channel = input_dim[0]
    self.out_channel = out_channel if out_channel is not None else self.in_channel * 2  # if not passed it is inferred from in_channel
    self.out_h = input_dim[1] // 2
    self.latent_dim = latent_dim
    
    self.downscaler = nn.Sequential(
      nn.Conv2d(in_channels=self.in_channel, out_channels=self.out_channel, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(self.out_channel),
      nn.ReLU(),
      nn.Conv2d(in_channels=self.out_channel, out_channels=self.out_channel, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(self.out_channel),
      nn.ReLU(),
      nn.Conv2d(in_channels=self.out_channel, out_channels=self.out_channel, kernel_size=4, stride=2, padding=1),
      nn.BatchNorm2d(self.out_channel),
      nn.ReLU()
    )
    
    self.mu = nn.Linear(self.out_channel*self.out_h*self.out_h, latent_dim)
    self.logvar = nn.Linear(self.out_channel*self.out_h*self.out_h, latent_dim)

  def forward(self, x:torch.Tensor):
    x = self.downscaler(x)
    mu = self.mu(x.view(x.shape[0], -1))
    logvar = self.logvar(x.view(x.shape[0], -1))
    return x, mu, logvar
  
class DecoderBlock(nn.Module):
  def __init__(
    self,
    input_dim:tuple,
    latent_dim:int
  ):
    super().__init__()
    
    self.in_channel = input_dim[0]
    self.out_channel = self.in_channel // 2  # if not passed it is inferred from in_channel
    self.in_h = input_dim[1]
    self.latent_dim = latent_dim
    
    self.projector = nn.Linear(latent_dim, self.in_channel*self.in_h*self.in_h)
    
    self.upscaler = nn.Sequential(
      nn.ConvTranspose2d(in_channels=self.in_channel, out_channels=self.out_channel, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(self.out_channel),
      nn.ReLU(),
      nn.ConvTranspose2d(in_channels=self.out_channel, out_channels=self.out_channel, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(self.out_channel),
      nn.ReLU(),
      nn.ConvTranspose2d(in_channels=self.out_channel, out_channels=self.out_channel, kernel_size=4, stride=2, padding=1),
      nn.BatchNorm2d(self.out_channel),
      nn.ReLU()
    )
    
  def forward(self, x:torch.Tensor, prev_generation:torch.Tensor):
    x = self.projector(x)
    x = x.view(x.shape[0], -1, self.in_h, self.in_h) + prev_generation
    x = self.upscaler(x)
    return x
    
    
class MultiVae(nn.Module):
  def __init__(self):
    super().__init__()
    
    self.encoder_1 = EncoderBlock(input_dim=(3,64,64), latent_dim=512, out_channel=64)
    self.encoder_2 = EncoderBlock(input_dim=(64,32,32), latent_dim=256)
    self.encoder_3 = EncoderBlock(input_dim=(128,16,16), latent_dim=128)
    
    self.decoder_1 = DecoderBlock(input_dim=(128,16,16), latent_dim=512)
    self.decoder_2 = DecoderBlock(input_dim=(256,8,8), latent_dim=256)
    self.decoder_3 = DecoderBlock(input_dim=(512,4,4), latent_dim=128)
    
    self.generator = nn.Sequential(
      nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, padding=1),
      nn.Sigmoid()
    )
    
    self.residual_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=2, padding=1)
    self.residual_2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1)
    
    
  def sample_latent_vector(self, mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return mu + eps * std
  
  def encode(self, x:torch.Tensor):
    x_new, mu1, logvar1 = self.encoder_1(x)
    x = self.residual_1(x)
    x_new, mu2, logvar2 = self.encoder_2(x_new + x)
    x = self.residual_2(x)
    x_new, mu3, logvar3 = self.encoder_3(x_new + x)
    
    return mu1, mu2, mu3, logvar1, logvar2, logvar3
  
  def decode(self, 
    mu1:torch.Tensor,
    mu2:torch.Tensor,
    mu3:torch.Tensor,
    logvar1:torch.Tensor,
    logvar2:torch.Tensor,
    logvar3:torch.Tensor
  ):
    latent_1 = self.sample_latent_vector(mu1, logvar1)
    latent_2 = self.sample_latent_vector(mu2, logvar2)
    latent_3 = self.sample_latent_vector(mu3, logvar3)
    
    x = self.decoder_3(latent_3, 0)
    x = self.decoder_2(latent_2, x)
    x = self.decoder_1(latent_1, x)
    
    return self.generator(x)
  
  def forward(self, x:torch.Tensor):
    mu1, mu2, mu3, logvar1, logvar2, logvar3 = self.encode(x)
    return self.decode(mu1, mu2, mu3, logvar1, logvar2, logvar3), mu1, mu2, mu3, logvar1, logvar2, logvar3
  
  def KL(self, x, mu, logvar):
    return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)
  
  def loss(self, x, recon_x, mu1, mu2, mu3, logvar1, logvar2, logvar3):
    recon_loss = F.mse_loss(x, recon_x, reduction='sum') / x.size(0)
    kl1 = self.KL(x, mu1, logvar1)
    kl2 = self.KL(x, mu2, logvar2)
    kl3 = self.KL(x, mu3, logvar3)
    return recon_loss + kl1 + kl2 + kl3, recon_loss, kl1, kl2, kl3

In [7]:
class plainVAE(nn.Module):
    def __init__(self, latent_dim: int = 128):
        super(plainVAE, self).__init__()
        self.latent_dim = latent_dim

        # Encoder: input 3x128x128 -> latent mean and log variance
        self.encoder = nn.Sequential(
            # Layer 1: 3x128x128 -> 32x64x64 # Should I use bigger images?
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),

            # Layer 2: 32x64x64 -> 64x32x32 # Should I use bigger images?
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            # Layer 3: 64x32x32 -> 128x16x16
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            # Layer 4: 128x16x16 -> 256x8x8
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            # # Layer 5: 256x8x8 -> 512x4x4
            # nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            # nn.BatchNorm2d(512),
            # nn.ReLU(inplace=True),
        )

        self.fc_mu = nn.Linear(512 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(512 * 4 * 4, latent_dim)

        self.decoder_fc = nn.Linear(latent_dim, 512 * 4 * 4)
        self.decoder = nn.Sequential(
            # Layer 1: 512*4*4 -> 256*8*8
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            # Layer 2: 256*8*8 -> 128*16*16
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            # Layer 3: 128*16*16 -> 64*32*32
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            # # Layer 4: 64*32*32 -> 32*64*64
            # nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),

            # Layer 4: 32*64*64 -> 3*128*128
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid(),
        )

    def encode(self, x):
        h = self.encoder(x)
        h = h.view(h.size(0), -1)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
    
    def reparameterization(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h = self.decoder_fc(z)
        h = h.view(h.size(0), 512, 4, 4)
        return self.decoder(h)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterization(mu, logvar)
        x_reconstructed = self.decode(z)
        return x_reconstructed, mu, logvar

    def loss(self, x, recon_x, mu, logvar, beta: float = 1.0):
        recon_loss = F.mse_loss(x, recon_x, reduction='sum') / x.size(0)
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)

        return recon_loss + beta * kl, recon_loss, kl

In [8]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),         # this ensures values ∈ [0,1]
])

In [ ]:
# VOICEPORTRAIT dataset

data_root = "/kaggle/input/vocalportrait/mavceleb_v1_train_cropped/mavceleb_v1_train_cropped/faces"
dataset = VocalPortraitDataset(data_root, transform)

In [9]:
# CELEBVOICE dataset

data_root = "/kaggle/input/cropped-celeba"
dataset = ImageFolder(root=data_root, transform=transform)

In [10]:
lengths = [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)]
train_dataset, test_dataset = random_split(dataset, lengths)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    pin_memory=True,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    pin_memory=True,
)

In [11]:
# ----------------------------
# 3) Instantiate model, optimizer, device
# ----------------------------
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = plainVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [111]:
model.load_state_dict(torch.load("/kaggle/working/vae_epoch5.pth"))

<All keys matched successfully>

In [ ]:
# ----------------------------
# 4) Training loop
# ----------------------------
num_epochs = 10
log_interval = 100   # how many batches between printouts

resume = False
checkpoint_path = "epoch{i}.pth"  # or whichever epoch you want

start_epoch = 1
if resume and os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Resumed model from {checkpoint_path}")
    # If you also saved the optimizer state, load it too:
    # optimizer.load_state_dict(torch.load("optimizer_epoch5.pth"))
    start_epoch = int(checkpoint_path.split("epoch")[1].split(".")[0]) + 1

for epoch in range(start_epoch, num_epochs + 1):
    model.train()
    running_recon_loss = 0.0
    running_kl_loss = 0.0
    running_total_loss = 0.0

    for batch_idx, (imgs, _) in enumerate(train_dataloader, start=1):
        imgs = imgs.to(device)   # shape (B, 3, 128, 128)
        optimizer.zero_grad()

        recon_imgs, mu, logvar = model(imgs)
        loss, recon_loss, kl_loss = model.loss(imgs, recon_imgs, mu, logvar, beta=0.1 * epoch)
        loss.backward()
        optimizer.step()

        running_total_loss += loss.item()
        running_recon_loss += recon_loss.item()
        running_kl_loss += kl_loss.item()

        if batch_idx % log_interval == 0:
            avg_total = running_total_loss / log_interval
            avg_recon = running_recon_loss / log_interval
            avg_kl    = running_kl_loss / log_interval
            print(f"Epoch [{epoch}/{num_epochs}]  "
                  f"Batch [{batch_idx}/{len(train_dataloader)}]  "
                  f"Total Loss: {avg_total:.3f}  "
                  f"Reconstruction: {avg_recon:.3f}  "
                  f"KL: {avg_kl:.3f}")
            running_total_loss = 0.0
            running_recon_loss = 0.0
            running_kl_loss    = 0.0

    # At the end of each epoch, you can save a checkpoint:
    checkpoint_path = f"vae_epoch{epoch}.pth"
    torch.save(model.state_dict(), checkpoint_path.format({"i": epoch}))

    # (Optional) Also: generate a few samples from N(0,I) and save to disk
    model.eval()
    with torch.no_grad():
        sample_z = torch.randn(64, model.latent_dim).to(device)
        sample_imgs = model.decode(sample_z)   # (64, 3, 128, 128)
        # Save a grid of 64 samples as a single image
        save_image(sample_imgs.cpu(), f"sample_epoch{epoch}.png", nrow=8, normalize=True)

print("Training finished.")

Epoch [1/10]  Batch [100/2019]  Total Loss: 286.599  Reconstruction: 263.578  KL: 230.206
Epoch [1/10]  Batch [200/2019]  Total Loss: 166.644  Reconstruction: 149.903  KL: 167.414
Epoch [1/10]  Batch [300/2019]  Total Loss: 137.472  Reconstruction: 121.123  KL: 163.489
Epoch [1/10]  Batch [400/2019]  Total Loss: 120.266  Reconstruction: 103.517  KL: 167.489
Epoch [1/10]  Batch [500/2019]  Total Loss: 109.660  Reconstruction: 93.210  KL: 164.509
Epoch [1/10]  Batch [600/2019]  Total Loss: 101.868  Reconstruction: 85.898  KL: 159.703
Epoch [1/10]  Batch [700/2019]  Total Loss: 95.382  Reconstruction: 79.532  KL: 158.500
Epoch [1/10]  Batch [800/2019]  Total Loss: 90.599  Reconstruction: 74.754  KL: 158.449
Epoch [1/10]  Batch [900/2019]  Total Loss: 87.149  Reconstruction: 71.465  KL: 156.838
Epoch [1/10]  Batch [1000/2019]  Total Loss: 82.605  Reconstruction: 67.077  KL: 155.278
Epoch [1/10]  Batch [1100/2019]  Total Loss: 79.883  Reconstruction: 64.409  KL: 154.742
Epoch [1/10]  Batch 

In [ ]:
model.eval()
for batch_idx, (imgs, _) in enumerate(test_dataloader, start=1):
    with torch.no_grad():
        imgs = imgs.to(device)
        sample_imgs, _, _ = model(imgs)   # (64, 3, 128, 128)
        # Save a grid of 64 samples as a single image
        save_image(sample_imgs.cpu(), f"sample_epoch_test.png", nrow=8, normalize=True)
    break